In [3]:
import pandas as pd
import psycopg2

class PostgresConnection:
    def __init__(self, dbname, user, password, host='localhost', port='5432'):
        self.dbname = dbname
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.conn = None
        self.cursor = None

    def connect(self):
        try:
            self.conn = psycopg2.connect(
                dbname=self.dbname,
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port
            )
            self.cursor = self.conn.cursor()
            print("Connected to PostgreSQL database!")
        except Exception as e:
            print(f"Error: {e}")

    def create_table(self):
        create_table_query = """
        CREATE TABLE IF NOT EXISTS telegram_post (
            id SERIAL PRIMARY KEY,
            date DATE NOT NULL,
            post_link TEXT NOT NULL,
            views TEXT,
            post_hour TIME,
            keyword TEXT,
            time_of_day TEXT
        );
        """
        try:
            self.cursor.execute(create_table_query)
            self.conn.commit()
            print("Table 'telegram_post' created successfully!")
        except Exception as e:
            print(f"Error creating table: {e}")
            self.conn.rollback()

    def insert_data(self, data):
        if self.conn is not None:
            try:
                for row in data:
                    insert_query = """
                    INSERT INTO telegram_post (date, post_link, views, post_hour, keyword, time_of_day)
                    VALUES (%s, %s, %s, %s, %s, %s);
                    """
                    self.cursor.execute(insert_query, row)
                self.conn.commit()
                print("Data inserted successfully!")
            except Exception as e:
                print(f"Error inserting data: {e}")
                self.conn.rollback()

    def close_connection(self):
        if self.conn is not None:
            self.cursor.close()
            self.conn.close()
            print("Connection closed.")

def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

def extract_attributes(row):
    keywords = [
        '#bank', '#CBE', '#BOA', '#GBE', '#CBO', '#ABAY', '#ABSC', '#ABYS', '#DASH', '#ENAT', '#BUNA',
        '#Commercial Bank of Ethiopia', '#bankofabyssinia', '#GlobalBankEthiopia', '#BoAEth', '#Coopbank', '#berhanbank'
    ]
    message = row.get('message', '')
    keyword_found = [kw for kw in keywords if kw in message]
    keyword_found_str = ', '.join(keyword_found) if keyword_found else None
    post_id = row.get('id', '')
    post_link = f"https://t.me/tikvahethiopia/{post_id}"
    views = row.get('views', '')
    date = pd.to_datetime(row['date']).date() if 'date' in row else ''
    post_hour = pd.to_datetime(row['date']).time() if 'date' in row else ''
    hour = int(post_hour.hour) if post_hour else 0
    time_of_day = get_time_of_day(hour)
    return [date, post_link, views, post_hour, keyword_found_str, time_of_day]

# Load the CSV data
df = pd.read_csv('./telegram_channel_data.csv')

# Apply the function to the DataFrame
extracted_data_list = df.apply(extract_attributes, axis=1).values.tolist()

# Filter rows that contain "https://t.me/tikvahethiopia/" in the post_link column
filtered_data = [row for row in extracted_data_list if 'https://t.me/tikvahethiopia/' in row[1]]

# Connect to PostgreSQL and create table
db = PostgresConnection(dbname='bank', user='postgres', password='Mati@1993')
db.connect()
db.create_table()

# Insert extracted data into PostgreSQL
db.insert_data(filtered_data)

# Close the connection
db.close_connection()

Connected to PostgreSQL database!
Table 'telegram_post' created successfully!
Data inserted successfully!
Connection closed.
